In [1]:
import numpy as np
import pygame
import sys
import math
import random
from pygame.locals import *

#deklarasi variabel yang diperlukan
JML_BARIS = 6
JML_KOLOM = 7
KOTAK = 75
PLAYER1 = 1
PLAYER2 = 2

#deklarasi warna yang digunakan
BLUE = (15,61,38)
BLACK = (47,79,79)
RED = (255,99,71)
YELLOW = (0,139,139)


#fungsi untuk membuat papan permainan
def buat_papan():
    papan = np.zeros((JML_BARIS, JML_KOLOM), dtype=int)
    return papan

#fungsi untuk menentukan posisi setiap bagian yg dijatuhkan dalam papan
def isi_papan(papan, baris,kolom, isi):
    papan[baris][kolom] = isi

#fungsi untuk memeriksa jika kolom sudah penuh
def cek_penuh(papan, kolom):
    return papan[JML_BARIS-1][kolom] == 0

#fungsi untuk memeriksa apakah baris sudah terisi belum
def cek_visited(papan, kolom):
    for r in range(JML_BARIS):
        if papan[r][kolom] == 0:
            return r
        
#mencetak papan dengan index (0,0) berada pada kiri bawah
def cetak_papan(papan):
    print(np.flip(papan,0))
        
#fungsi untuk memeriksa jika menang
def cek_menang(papan, isi):
    #cek horizontal
    for c in range (JML_KOLOM-3):
        for r in range (JML_BARIS):
            if papan[r][c] == isi and papan[r][c+1] == papan[r][c+2] == papan[r][c+3] == isi:
                return True    
    #cek vertikal
    for c in range (JML_KOLOM):
        for r in range (JML_BARIS-3):
            if papan[r][c] == isi and papan[r+1][c] == papan[r+2][c] == papan[r+3][c] == isi:
                return True        
    #cek diagonal positif
    for c in range (JML_KOLOM-3):
        for r in range (JML_BARIS-3):
            if papan[r][c] == isi and papan[r+1][c+1] == papan[r+2][c+2] == papan[r+3][c+3] == isi:
                return True 
    #cek diagonal negatif
    for c in range (JML_KOLOM-3):
        for r in range (3,JML_BARIS):
            if papan[r][c] == isi and papan[r-1][c+1] == papan[r-2][c+2] == papan[r-3][c+3] == isi:
                return True 

#Mulai Ai

def nilai_pemain(window, isi):
	score = 0
	isi_lawan = PLAYER1
	if isi == PLAYER1:
		isi_lawan = PLAYER2
    

	if window.count(isi) == 4:
	    score += 100
	elif window.count(isi) == 3 and window.count(0) == 1:
	    score += 5
	elif window.count(isi) == 2 and window.count(0) == 2:
	    score += 2
	if window.count(isi_lawan) == 3 and window.count(0) == 1:
		score -= 4

	return score


def cek_nilai(papan, isi):

    score = 0

    #score horizontal
    for r in range(JML_BARIS):
        array_baris = [int (i) for i in list(papan[r,:])]
        #cek sampai ada empat isi tiap baris 
        for c in range(JML_KOLOM-3):
            window = array_baris[c:c+4]
            score += nilai_pemain(window,isi)
    #score vertikal
    for c in range(JML_KOLOM):
        array_kolom = [int (i) for i in list(papan[:,c])]
        for r in range(JML_BARIS-3):
            window = array_kolom[r:r+4]
            score += nilai_pemain(window,isi)
    #score diagonal
    for r in range(JML_BARIS-3):
        for c in range(JML_KOLOM-3):
            window = [papan[r+i][c+i] for i in range(4)]
            score += nilai_pemain(window, isi)
    for r in range(JML_BARIS-3):
        for c in range(JML_KOLOM-3):
            window = [papan[r+3-i][c+i] for i in range(4)]
            score += nilai_pemain(window, isi)
    return score
    #score tengah kolom
    center_array = [int(i) for i in list(papan[:, JML_KOLOM//2])]
    center_count = center_array.count(isi)
    score += center_count*3

def get_cek_penuh(papan):
    cek = []
    for kolom in range(JML_KOLOM):
        if cek_penuh(papan,kolom):
            cek.append(kolom)
    return cek

def best_move (papan, isi):
    
    cek = get_cek_penuh(papan)
    best_score = 0
    best_kolom = random.choice(cek)
    for kolom in cek:
        baris = cek_visited(papan, kolom)
        temp_papan = papan.copy()
        isi_papan(temp_papan, baris, kolom, isi)
        score = cek_nilai(temp_papan, isi)
        if score > best_score:
            best_score = score
            best_kolom = kolom
    return best_kolom

def is_terminal(papan):
    return cek_menang(papan, PLAYER1) or cek_menang(papan, PLAYER2) or len(get_cek_penuh(papan)) == 0


def alpha_beta(papan, depth, alpha, beta, maximizingPlayer):
    loc = get_cek_penuh(papan)
    terminal = is_terminal(papan)

    if depth == 0 or terminal:
         if terminal:
             if cek_menang(papan, PLAYER1):
                  return (None, -math.inf)
             elif cek_menang(papan, PLAYER2):
                  return (None, math.inf)
             else: #Game sudah selesai, tidak ada pergerakan dari pemain
                  return (None, 0)
         else: #depth == 0
             return (None, cek_nilai(papan, PLAYER2))

    if maximizingPlayer: #memaksimalkan permainan AI
        val = -math.inf
        kolom = random.choice(loc)
        for k in loc:
            baris = cek_visited(papan, k)
            papan_copy = papan.copy()
            isi_papan(papan_copy, baris, k, PLAYER2)
            new_score = alpha_beta(papan_copy, depth-1, alpha, beta, False)[1]
            if new_score > val:
               val = new_score
               kolom = k
            alpha = max(alpha, val)
            if alpha >= beta:
               break
        return kolom, val

    else: #Meminimalkan permainan PLAYER
        val = math.inf
        kolom = random.choice(loc)
        for k in loc:
            baris = cek_visited(papan, k)
            papan_copy = papan.copy()
            isi_papan(papan_copy, baris, k, PLAYER1)
            new_score = alpha_beta(papan_copy, depth-1, alpha, beta, True)[1]
            if new_score < val:
               val = new_score
               kolom = k
               betha = min(beta, val)
            if alpha >= beta:
               break
        return kolom, val

#fungsi untuk menampilkan dengan pygame        
def draw_board(papan):
    for c in range(JML_KOLOM):
        for r in range (JML_BARIS):
            pygame.draw.rect(screen, BLUE,(c*KOTAK,r*KOTAK+KOTAK, KOTAK, KOTAK))
            pygame.draw.circle(screen, BLACK,(int(c*KOTAK+KOTAK/2),int(r*KOTAK+KOTAK+KOTAK/2)), radius )
    for c in range(JML_KOLOM):
        for r in range (JML_BARIS):
            if papan[r][c] == PLAYER1:
                pygame.draw.circle(screen, RED,(int(c*KOTAK+KOTAK/2),int(tinggi-(r*KOTAK+KOTAK/2)) ), radius )
            elif papan[r][c] == PLAYER2:
                pygame.draw.circle(screen, YELLOW,(int(c*KOTAK+KOTAK/2),int( tinggi - (r*KOTAK+KOTAK/2) )), radius )
    pygame.display.update()
    

lebar = JML_KOLOM * KOTAK
tinggi = (JML_BARIS+1)* KOTAK 
size = (lebar,tinggi)
radius = int (KOTAK/2 - 5 )

clock = pygame.time.Clock()
pygame.init()
pygame.display.set_caption('Connect Four Game')
screen = pygame.display.set_mode(size)
font = pygame.font.SysFont("Century Gothic",50)
tulisan = pygame.font.SysFont("Arial",50)
tulisan2 = pygame.font.SysFont("Arial",20)


def draw_text(text, font, color, surface, x, y):
    textobj = font.render(text, 1, color)
    textrect = textobj.get_rect()
    textrect.topleft = (x, y)
    surface.blit(textobj, textrect)


def options(click):
    while True:

        screen.fill(BLACK)
        draw_text('Play Again?', font, RED , screen, 120, 100)
        mx, my = pygame.mouse.get_pos()
        button1 = pygame.Rect(150, 200, 200, 50)
        button2 = pygame.Rect(150, 300, 200, 50)
        button3 = pygame.Rect(150, 400, 200, 50)
       
        if button1.collidepoint((mx, my)):
            if click:
                game_with_AI()
        if button2.collidepoint((mx, my)):
            if click:
                game_two_players()
        if button3.collidepoint((mx, my)):
            if click:
                pygame.quit()
                sys.exit()
        pygame.draw.rect(screen, BLUE, button1)
        pygame.draw.rect(screen, BLUE, button2)

        click = False
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True

        draw_text('One Player', tulisan2, RED, screen, 200, 210)
        draw_text('Two Players', tulisan2, RED, screen, 200, 310)
        draw_text('QUIT', tulisan2, RED, screen, 230, 410)
        pygame.display.update()
        clock.tick(60)  
        

def main_menu(click):
    while True:

        screen.fill(BLACK)
        draw_text('WELCOME TO', tulisan2, RED, screen, 200, 80)
        draw_text('CONNECT 4 GAME', font, RED, screen, 30, 100)
        draw_text('Lets Play!!!', tulisan2, RED, screen, 215, 170)
        
        mx, my = pygame.mouse.get_pos()
        one = pygame.Rect(150, 200, 200, 50)
        two = pygame.Rect(150, 300, 200, 50)
        
        if one.collidepoint((mx, my)):
            if click :
                game_with_AI()
        if two.collidepoint((mx, my)):
            if click:
                game_two_players()
        pygame.draw.rect(screen, BLUE, one)
        pygame.draw.rect(screen, BLUE, two)
        
        click = False
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True

        draw_text('One Player', tulisan2, RED, screen, 200, 210)
        draw_text('Two Players', tulisan2, RED, screen, 200, 310)
        
        pygame.display.update()
        clock.tick(60)
        
def game_with_AI() :
    click = False
    running = True
    turn = 0    
    while running :
        draw_text('Permainan Dimulai !', tulisan, (255, 255, 255), screen, 50, 10)
        papan = buat_papan()
        cetak_papan(papan)
        draw_board(papan)
        
        game_over = False 
        
        while not game_over :
            for event in pygame.event.get():
                
                #untuk keluar dari tampilan
                if event.type == pygame.QUIT:
                    sys.exit()

                #yang ditampilkan pada gerakan mouse
                if event.type == pygame.MOUSEMOTION:
                    pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))
                    posx = event.pos[0]
                    if turn == 0:
                        pygame.draw.circle(screen, RED, (posx, int(KOTAK/2)), radius )
                pygame.display.update()


                if event.type == pygame.MOUSEBUTTONDOWN:
                    pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))

                    if turn == 0:
                        posx = event.pos[0]
                        kolom = int(math.floor(posx/KOTAK))          
                        if cek_penuh(papan, kolom):
                            baris = cek_visited(papan,kolom)
                            isi_papan(papan, baris,kolom, PLAYER1)
                            if cek_menang(papan, PLAYER1):
                                label = tulisan.render("Pemain menang!!!", 1, RED )
                                screen.blit(label,(40,10))
                                game_over = True
                            turn += 1
                            turn = turn%2
                            cetak_papan(papan)
                            draw_board(papan)
                        else:
                            pygame.quit()
            
            if turn == 1 and not game_over:  
                #kolom = random.randint(0,JML_KOLOM-1)
                #kolom = best_move(papan, AI)
                kolom, alpha_beta_score = alpha_beta(papan, 3, -math.inf, math.inf, True)
                if cek_penuh(papan, kolom):
                    baris = cek_visited(papan,kolom)
                    isi_papan(papan, baris,kolom, PLAYER2)
                    if cek_menang(papan, PLAYER2):
                        label = tulisan.render("Komputer menang!!!", 1, YELLOW )
                        screen.blit(label,(40,10))
                        game_over = True
                        
                    cetak_papan(papan)
                    draw_board(papan)
                    turn += 1
                    turn = turn%2
                    
            if game_over:
                pygame.time.wait(3000)
                options(click)
    
    pygame.display.update()
    clock.tick(60)

def game_two_players():
    click = False
    running = True
    turn = 0    
    while running :
        draw_text('Permainan Dimulai !', tulisan, (255, 255, 255), screen, 50, 10)
        papan = buat_papan()
        cetak_papan(papan)
        draw_board(papan)
        
        game_over = False 
        
        while not game_over :
            for event in pygame.event.get():
                
                #untuk keluar dari tampilan
                if event.type == pygame.QUIT:
                    sys.exit()

                #yang ditampilkan pada gerakan mouse
                if event.type == pygame.MOUSEMOTION:
                    pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))
                    posx = event.pos[0]
                    if turn == 0:
                        pygame.draw.circle(screen, RED, (posx, int(KOTAK/2)), radius )
                    else:
                        pygame.draw.circle(screen, YELLOW, (posx, int(KOTAK/2)), radius )
                pygame.display.update()
                
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pygame.draw.rect(screen, BLACK,(0,0, lebar, KOTAK))
                    
                    #Meminta input dari player 1
                    if turn == 0:
                        posx = event.pos[0]
                        kolom = int(math.floor(posx/KOTAK))     

                        if cek_penuh(papan, kolom):
                            baris = cek_visited(papan,kolom)
                            isi_papan(papan, baris,kolom, PLAYER1)

                            if cek_menang(papan, PLAYER1):
                                label = tulisan.render("Pemain 1 menang!!!", 1, RED )
                                screen.blit(label,(40,10))
                                game_over = True


                    #Meminta input dari player2
                    else:  
                        posx = event.pos[0]
                        kolom = int(math.floor(posx/KOTAK))

                        if cek_penuh(papan, kolom):
                            baris = cek_visited(papan,kolom)
                            isi_papan(papan, baris,kolom, PLAYER2)

                            if cek_menang(papan, PLAYER2):
                                label = tulisan.render("Pemain 2 menang!!!", 1, YELLOW )
                                screen.blit(label,(40,10))
                                game_over = True

                    cetak_papan(papan)
                    draw_board(papan)

                    #Untuk update giliran permainan
                    turn += 1
                    turn = turn%2
                    
                    if game_over:
                        pygame.time.wait(3000)
                        options(click)
    
    pygame.display.update()
    clock.tick(60)

click = False
main_menu(click)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 2 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 1 0 0]
 [0 0 0 2 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 2 1 0 0]
 [0 0 0 2 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 1 0 0]
 [0 0 0 2 1 0 0]
 [0 0 0 2 1 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 2 1 0 0]
 [0 0 0 2 1 0 0]
 [0 0 0 2 1 0 0]]


SystemExit: 

C:\Users\windows\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
